# Predicting Stock Market Using Recurrent Neural Network (RNN) 

## Importing Libraries

In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

%matplotlib widget

Using TensorFlow backend.


## Loading Dataset

In [2]:
df = pd.read_csv('Tesla.csv', index_col=0, parse_dates=True)
df

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2010-06-29,19.000000,25.000000,17.540001,23.889999,18766300,23.889999
2010-06-30,25.790001,30.420000,23.299999,23.830000,17187100,23.830000
2010-07-01,25.000000,25.920000,20.270000,21.959999,8218800,21.959999
2010-07-02,23.000000,23.100000,18.709999,19.200001,5139800,19.200001
2010-07-06,20.000000,20.000000,15.830000,16.110001,6866900,16.110001
2010-07-07,16.400000,16.629999,14.980000,15.800000,6921700,15.800000
2010-07-08,16.139999,17.520000,15.570000,17.459999,7711400,17.459999
2010-07-09,17.580000,17.900000,16.549999,17.400000,4050600,17.400000
2010-07-12,17.950001,18.070000,17.000000,17.049999,2202500,17.049999


## Visualizing the dataset

In [3]:
plt.figure(figsize=(8,5))
plt.title('Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=15)
plt.ylabel('Close Price USD ($)', fontsize=15)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Data Preprocessing

### Determining the used data

In [4]:
data = df.filter(['Close']) # Data with only "Close" column

dataset = data.values # Converting data to array

train_data_len = math.ceil(len(dataset)*0.8) # Determining the lenght of training set
                                             # With this, we can also determine the test set in the next line
train_data_len

1354

### Scaling the dataset

In [5]:
scaler = MinMaxScaler(feature_range=(0,1))
data_scaled = scaler.fit_transform(dataset)
data_scaled

array([[0.02993635],
       [0.02971433],
       [0.02279455],
       ...,
       [0.88784039],
       [0.91122698],
       [0.9091918 ]])

### Creating the training set

In [6]:
train_data = data_scaled[0:train_data_len , :]

# Splitting training data into x_train and y_train
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])
    if i<=61:
        print(x_train)
        print(y_train)
        print()

[array([0.02993635, 0.02971433, 0.02279455, 0.01258141, 0.00114713,
       0.        , 0.00614268, 0.00592066, 0.00462551, 0.00865897,
       0.01494967, 0.01513469, 0.01791   , 0.02260953, 0.01665186,
       0.01635583, 0.01924215, 0.02031528, 0.01905714, 0.01757696,
       0.01820603, 0.01683689, 0.01531972, 0.01894612, 0.02275755,
       0.02020426, 0.01720693, 0.01402457, 0.01406157, 0.01195234,
       0.00777087, 0.00666075, 0.00932504, 0.01102724, 0.01239639,
       0.01099023, 0.01106424, 0.01221137, 0.01602279, 0.01258141,
       0.0151717 , 0.01461664, 0.01443162, 0.01506069, 0.01361752,
       0.01720693, 0.01946418, 0.01942717, 0.01753997, 0.01887211,
       0.01816903, 0.01617081, 0.01820603, 0.01968621, 0.02286856,
       0.01902013, 0.01639284, 0.01946418, 0.01839106, 0.01506069])]
[0.01391355415474397]

[array([0.02993635, 0.02971433, 0.02279455, 0.01258141, 0.00114713,
       0.        , 0.00614268, 0.00592066, 0.00462551, 0.00865897,
       0.01494967, 0.01513469, 0.01

### Converting x_train and y_train to numpy arrays

In [7]:
x_train, y_train = np.array(x_train), np.array(y_train)

### Reshaping the data

In [8]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(1294, 60, 1)

## Building the LSTM model

In [9]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

## Compiling the model

In [10]:
model.compile(optimizer='adam', loss='mean_squared_error')

## Training the model

In [11]:
model.fit(x_train, y_train, batch_size=1, epochs=1)


Epoch 1/1
1294/1294 [==============================] - 96s 74ms/step - loss: 0.0031


## Creating testing data

In [12]:
test_data = data_scaled[train_data_len - 60:, :]

x_test = []
y_test = dataset[train_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

In [13]:
x_test = np.array(x_test)

In [14]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

## Predicting the testing data

In [15]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

## Getting the root mean square error

In [16]:
rmse = np.sqrt(np.mean(predictions-y_test)**2)
rmse

4.186520680502266

## Visualizing the predicted data

In [17]:
train = data[:train_data_len]
valid = data[train_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(10,5))
plt.title('Model')
plt.xlabel('Date', fontsize=15)
plt.ylabel('Close Price USD ($)', fontsize=15)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
valid

,Close,Predictions
Date,,
2015-11-12,212.940002,227.800156
2015-11-13,207.190002,226.844910
2015-11-16,214.309998,224.857208
2015-11-17,214.000000,223.214005
2015-11-18,221.070007,221.833817
2015-11-19,221.800003,221.448288
2015-11-20,220.009995,221.699570
2015-11-23,217.750000,222.126999
2015-11-24,218.250000,222.416260
